# Exercise 10 : Gluon Path Integrals

### Function to generate an array of SU(3) matrices and their inverses to use in the monte-carlo process

See Gattringer and Lang page 83. Although they say you should choose random numbers between $(-0.5, 0.5)$ this leads to negative numbers on the diagonal for the generated su3 matrices, these are not closer to the identity for smaller epsilon (as -.999 is generated for epsilon = 0.01 and -.075 is generated for epsilon = 0.5, the first value is obviously further away from 1 even though the epsilon is smaller). Hence smaller epsilon does not give a higher acceptance which is required to tune it for 50%

So I will use random numbers from $(0,0.5)$ I tested the expectation values obtained from $(0,0.5)$ and $(-0.5,0.5)$ and they are the same. After emailing lang, only need 3 random numbers and r[0] = sqrt(xxx) without sign function

In [15]:
import numpy as np
import os
from functools import partial
from math import factorial
from re import split
from time import time


def generate_su2(epsilon):
    
    r_random_numbers = np.random.uniform(0, 0.5, (4))
    r = np.empty((4))
    
    r[1:] = epsilon * r_random_numbers[1:] / np.linalg.norm(r_random_numbers[1:])
    r[0] = np.sign(r_random_numbers[0]) * np.sqrt(1-epsilon**2)
    
    r11 = r[0] + r[3]*1j
    r12 = r[1]*1j + r[2]
    r21 = r[1]*1j - r[2]
    r22 = r[0] - r[3]*1j
    
    return np.array([[r11, r12], [r21, r22]])
    
    
def generate_su3_array(n, epsilon):

    su3_array = np.empty((2*n, 3, 3), dtype = 'complex128')
    
    for i in range(n):

        R_su3 = np.identity(3, dtype=complex)
        S_su3 = np.identity(3, dtype=complex)
        T_su3 = np.identity(3, dtype=complex)

        R_su3[:2, :2] = generate_su2(epsilon)
        S_su3[0:3:2, 0:3:2] = generate_su2(epsilon)
        T_su3[1:, 1:] = generate_su2(epsilon)

        X_su3 = np.dot(np.dot(R_su3, S_su3), T_su3)

        su3_array[2*i, :, :] = X_su3
        su3_array[2*i+1, :, :] = X_su3.conj().T
        
    return su3_array

### Classes for the lattice and the site at each lattice point.

Site value is just a scalar here as this is a simple gauge simulation. The fundamental data structure of the lattice is a numpy array of site objects which have four links initialised to the identity matrix.

In [16]:
def generate_site():
    
    link_value = np.identity(3, dtype = 'complex128')
    link = np.tile(link_value, (4, 1, 1))
    
    return link
        

def generate_lattice(n_points):

    volume = np.append(np.repeat(n_points, 4), (4,3,3))
    grid = np.empty(volume, dtype = 'complex128')
        
    for t in range(n_points):
        for x in range(n_points):
            for y in range(n_points):
                for z in range(n_points):
                    grid[t, x, y, z] = generate_site()
                        
    return grid

def link(lattice, coords, mu):
        
    n_points = lattice.shape[0]
        
    return lattice[coords[0] % n_points, coords[1] % n_points, coords[2] % n_points, coords[3] % n_points, mu, : , :]

### Function which computes the staple sum for a given site link

The function takes a coordinate array [t, x, y, z] for the lattice and a direction t = 0, x = 1, y = 2 or z = 3 and computes the sum of staples which include this link. This corresponds to the Wilson action. See Gattringer and Lang page 79 eqn (4.20).

In [17]:
def wilson_link_sum(lattice, coordinates,  mu, u_0):
    dimension = 4
    res = np.zeros((3,3), dtype = 'complex128')
    
    for nu in range(dimension):
        if nu != mu:
            
            mu_hat = np.zeros(dimension, dtype = int)
            nu_hat = np.zeros(dimension, dtype = int)
            
            mu_hat[mu] = 1
            nu_hat[nu] = 1
            
            #1x1 positive
            res += np.dot(np.dot(link(lattice, coordinates + mu_hat, nu), 
                                 link(lattice, coordinates + nu_hat, mu).conj().T),
                                 link(lattice, coordinates, nu).conj().T)
            #1x1 negative
            res += np.dot(np.dot(link(lattice, coordinates + mu_hat - nu_hat, nu).conj().T, 
                                 link(lattice, coordinates - nu_hat, mu).conj().T), 
                                 link(lattice, coordinates - nu_hat, nu))
        
    return res / u_0**4


def improved_link_sum(lattice, coordinates, mu, u_0):
    #Rectangular Improvement
    
    dimension = 4
    res_rec = np.zeros((3,3), dtype = 'complex128')
    
    for nu in range(dimension):
        if nu != mu:
            
            mu_hat = np.zeros(dimension, dtype = int)
            nu_hat = np.zeros(dimension, dtype = int)
            
            mu_hat[mu] = 1
            nu_hat[nu] = 1
            
    
            link_00_nu = link(lattice, coordinates, nu)
            link_10_mu = link(lattice, coordinates + mu_hat, mu)
            link_10_nu = link(lattice, coordinates + mu_hat, nu)
            link_11_mu = link(lattice, coordinates + nu_hat + mu_hat, mu)
            link_11_nu = link(lattice, coordinates + nu_hat + mu_hat, nu)
            link_01_mu = link(lattice, coordinates + nu_hat, mu)
            link_01_nu = link(lattice, coordinates + nu_hat, nu)
            link_20_nu = link(lattice, coordinates + 2*mu_hat, nu)
            link_02_mu = link(lattice, coordinates + 2*nu_hat, mu) 
            
            link_1n1_mu = link(lattice, coordinates + mu_hat - nu_hat, mu)
            link_1n1_nu = link(lattice, coordinates + mu_hat - nu_hat, nu)
            link_0n1_mu = link(lattice, coordinates - nu_hat, mu)
            link_0n1_nu = link(lattice, coordinates - nu_hat, nu)
            link_0n2_mu = link(lattice, coordinates - 2*nu_hat, mu)
            link_0n2_nu = link(lattice, coordinates - 2*nu_hat, nu)
            link_2n1_nu = link(lattice, coordinates + 2*mu_hat - nu_hat, nu)
            link_1n2_nu = link(lattice, coordinates + mu_hat - 2*nu_hat, nu)
            
            #2x1 positive rectangle
            res_rec += np.dot(np.dot(np.dot(np.dot(link_10_mu,
                                                   link_20_nu),
                                                   link_11_mu.conj().T), 
                                                   link_01_mu.conj().T),
                                                   link_00_nu.conj().T)
            #2x1 negative rectangle
            res_rec += np.dot(np.dot(np.dot(np.dot(link_10_mu,
                                                   link_2n1_nu.conj().T),
                                                   link_1n1_mu.conj().T), 
                                                   link_0n1_mu.conj().T),
                                                   link_0n1_nu)
            #1x2 postive rectangle
            res_rec += np.dot(np.dot(np.dot(np.dot(link_10_nu,
                                                   link_11_nu),
                                                   link_02_mu.conj().T), 
                                                   link_01_nu.conj().T),
                                                   link_00_nu.conj().T)
            #1x2 negative rectangle
            res_rec += np.dot(np.dot(np.dot(np.dot(link_1n1_nu.conj().T,
                                                   link_1n2_nu.conj().T),
                                                   link_0n2_mu.conj().T), 
                                                   link_0n2_nu),
                                                   link_0n1_nu)
    
    res = 5 * wilson_link_sum(lattice, coordinates,  mu, u_0) / 3 - res_rec / u_0 **6 / 12
    
    return res       

### Update function for a single gauge link

The function takes a su3 matrix, coordinate array, direction and constant beta and preforms a single montecarlo step for a single gauge link

In [18]:
def update_link(lattice, link_sum, su3_matrix, coords, mu, beta, u_0, n_hits):
    
    accept = 0
    
    t = coords[0]
    x = coords[1]
    y = coords[2]
    z = coords[3]
    
    A = link_sum(lattice, coords,  mu, u_0)
    
    for i in range(n_hits):
    
        accept += 1
    
        gauge_link_old = np.copy(lattice[t, x, y, z, mu, : , :])
        
        lattice[t, x, y, z, mu, : , :] = np.dot(su3_matrix, gauge_link_old)
        
        diff = lattice[t, x, y, z, mu, : , :] - gauge_link_old
    
        deltaS = -1.0 * beta / u_0 * np.trace(np.dot(diff, A)).real
    
        r = np.minimum(1.0, np.exp(-1.0 * deltaS))
    
        if r < np.random.rand():
        
            lattice[t, x, y, z, mu, : , :] = gauge_link_old
            
            accept -= 1
    
    return accept / n_hits

### Function to update whole lattice

In [19]:
def update_lattice(lattice, link_sum, su3_set, beta, u_0, n_hits):
    n_points = lattice.shape[0]
    dimension = 4 
    acceptance = 0
    su3_set_length = len(su3_set)
    
    for t in range(n_points):
        for x in range(n_points):
            for y in range(n_points):
                for z in range(n_points):
                    for mu in range(dimension):
                        
                        su3_matrix = su3_set[np.random.randint(0, su3_set_length)]
                        acceptance += update_link(lattice, link_sum, su3_matrix, [t,x,y,z], mu, beta, u_0, n_hits)
                            
    return acceptance / ((dimension) * (n_points ** dimension))

### Ensemble Generation

Do individual hits count as updates for correlation? assumming they do and 50 correlation steps = 5 correlation x 10 hits

In [20]:
def config_save(configuration, action, beta, volume, k):
    
    filename = "gauge_configs/" + str(action) + "/beta_" + str(beta) + "/vol_" + str(volume) + "/" + str(k)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    np.save(filename, configuration, allow_pickle = True)
    print("Configuration " + str(k) + " saved")

def markov_chain(lattice, link_sum,  su3_set, beta, u_0, n_configs, n_corr, n_hits, thermalise = True, save = True):
    
    acceptance = 0
    volume = lattice.shape[:4]
    
    #Thermalise
    if thermalise == True:
        for i in range(5*n_corr):
            
            update_lattice(lattice, link_sum, su3_set, beta, u_0, n_hits)
    
    #Equilibrium Update
    for j in range(n_configs*n_corr):
        
        acceptance += update_lattice(lattice, link_sum, su3_set, beta, u_0, n_hits)
        
        if save == True:
            if j % n_corr == 0:
            
                ensemble_index = int(j/n_corr)
                config_save(lattice, action.__name__, beta, volume, ensemble_index)
    
    if n_configs != 0 and n_corr != 0:
    
        acceptance = acceptance / (n_configs*n_corr)       
    
    return acceptance

### Load from file

In [21]:
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]


def load_ensemble(action, beta, volume):
    #wilson_action or rectangular_action
    
    ensemble = []
    directory = "gauge_configs/" + str(action) + "/beta_" + str(beta)  + "/"
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in sorted(files, key = natural_key):
            ensemble.append(np.load(os.path.abspath(os.path.join(root, name)), allow_pickle=True))
                        
    return ensemble

### Wilson loop functions to calculate expectation values

In [22]:
def square_wilson_loop(lattice, coordinates, mu, nu):
    
    dimension = 4
    res = np.zeros((3,3), dtype = 'complex128')
    
    mu_hat = np.zeros(dimension, dtype = int)
    nu_hat = np.zeros(dimension, dtype = int)
            
    mu_hat[mu] = 1
    nu_hat[nu] = 1
    
    res = np.dot(np.dot(np.dot(link(lattice, coordinates,          mu), 
                               link(lattice, coordinates + mu_hat, nu)), 
                               link(lattice, coordinates + nu_hat, mu).conj().T), 
                               link(lattice, coordinates,          nu).conj().T)
        
    return np.trace(res).real / 3.0


def rectangular_wilson_loop(lattice, coordinates, mu, nu):
    
    dimension = 4
    res = np.zeros((3,3), dtype = 'complex128')
    
    mu_hat = np.zeros(dimension, dtype = int)
    nu_hat = np.zeros(dimension, dtype = int)
            
    mu_hat[mu] = 1
    nu_hat[nu] = 1
    
    link_00_mu = link(lattice, coordinates, mu)
    link_00_nu = link(lattice, coordinates, nu)
    link_10_mu = link(lattice, coordinates + mu_hat, mu)
    link_10_nu = link(lattice, coordinates + mu_hat, nu)
    link_11_mu = link(lattice, coordinates + nu_hat + mu_hat, mu)
    link_11_nu = link(lattice, coordinates + nu_hat + mu_hat, nu)
    link_01_mu = link(lattice, coordinates + nu_hat, mu)
    link_01_nu = link(lattice, coordinates + nu_hat, nu)
    link_20_nu = link(lattice, coordinates + 2*mu_hat, nu)
    link_02_mu = link(lattice, coordinates + 2*nu_hat, mu) 
            
            
    #2x1 positive rectangle
    res1 = np.dot(np.dot(np.dot(np.dot(np.dot(link_00_mu, 
                                              link_10_mu),
                                              link_20_nu),
                                              link_11_mu.conj().T), 
                                              link_01_mu.conj().T),
                                              link_00_nu.conj().T)
    #1x2 postive rectangle
    res2 = np.dot(np.dot(np.dot(np.dot(np.dot(link_00_mu, 
                                              link_10_nu),
                                              link_11_nu),
                                              link_02_mu.conj().T), 
                                              link_01_nu.conj().T),
                                              link_00_nu.conj().T)
            
    return (np.trace(res1).real + np.trace(res2).real) / 6

### Functions to average loop over the lattice and the ensemble

In [23]:
def lattice_average(lattice, wilson_loop_function):
    
    dimension = 4
    n_points = lattice.shape[0]
    
    unique_2d_loops_number = 6
    res = 0
    
    for t in range(n_points):
        for x in range(n_points):
            for y in range(n_points):
                for z in range(n_points):
                    for mu in range(dimension):
                        for nu in range(dimension):
                            if nu < mu:
                                res += wilson_loop_function(lattice, [t,x,y,z], mu, nu)
                                
    return res / ((n_points**dimension)*unique_2d_loops_number)


def gauge_average(lattice_ensemble, wilson_loop_function):

    func = partial(lattice_average, wilson_loop_function = wilson_loop_function)
    loop_ensemble = list(map(func, lattice_ensemble))
        
    return np.mean(loop_ensemble), np.std(loop_ensemble) / len(lattice_ensemble)

In [24]:
lattice = generate_lattice(n_points = 8)

su3_array = generate_su3_array(100, epsilon = 0.1)

t1a = time()
acceptance1 = markov_chain(lattice, wilson_link_sum, su3_array, beta = 5.5, u_0= 1, n_configs = 1, n_corr = 50, n_hits = 1, save = False)
t1b = time()

t1c = time()
wilson_square_loop = lattice_average(lattice, square_wilson_loop)
t1d = time()

t1e = time()
wilson_rectangular_loop = lattice_average(lattice, rectangular_wilson_loop)
t1f = time()

t2a = time()
acceptance2 = markov_chain(lattice, improved_link_sum, su3_array, beta = 1.719, u_0= 0.797, n_configs = 1, n_corr = 50, n_hits= 1, save = False)
t2b = time()

t2c = time()
improved_square_loop = lattice_average(lattice, square_wilson_loop)
t2d = time()

t2e = time()
improved_rectangular_loop = lattice_average(lattice, rectangular_wilson_loop)
t2f = time()


print("Wilson action acceptance: " + str(acceptance1) + " total time: " + str(t1b-t1a))
print("Wilson action square loop expectation " + str(wilson_square_loop) + " time: " + str(t1d-t1c))
print("Wilson action rectangular loop expectation " + str(wilson_rectangular_loop) + " time: " + str(t1f-t1e))

print("Improved action acceptance: " + str(acceptance2) + " total time: " + str(t2b-t2a))
print("Improved action square loop expectation " + str(improved_square_loop) + " time: " + str(t2d-t2c))
print("Improved action rectangular loop expectation " + str(improved_rectangular_loop) + " time: " + str(t2f-t2e))

del lattice

Wilson action acceptance: 0.489658203125 total time: 602.9789292812347
Wilson action square loop expectation 0.877738632244 time: 0.629753828048706
Wilson action rectangular loop expectation 0.798595060679 time: 1.529761791229248
Improved action acceptance: 0.41093994140625 total time: 1921.2175521850586
Improved action square loop expectation 0.905572299337 time: 0.5414674282073975
Improved action rectangular loop expectation 0.835114354974 time: 1.3294868469238281


In [25]:
lattice = generate_lattice(n_points = 8)

su3_array = generate_su3_array(100, epsilon = 0.1)

t1a = time()
acceptance1 = markov_chain(lattice, wilson_link_sum, su3_array, beta = 5.5, u_0= 1, n_configs = 1, n_corr = 50, n_hits = 5, save = False)
t1b = time()

t1c = time()
wilson_square_loop = lattice_average(lattice, square_wilson_loop)
t1d = time()

t1e = time()
wilson_rectangular_loop = lattice_average(lattice, rectangular_wilson_loop)
t1f = time()

t2a = time()
acceptance2 = markov_chain(lattice, improved_link_sum, su3_array, beta = 1.719, u_0= 0.797, n_configs = 1, n_corr = 50, n_hits= 5, save = False)
t2b = time()

t2c = time()
improved_square_loop = lattice_average(lattice, square_wilson_loop)
t2d = time()

t2e = time()
improved_rectangular_loop = lattice_average(lattice, rectangular_wilson_loop)
t2f = time()


print("Wilson action acceptance: " + str(acceptance1) + " total time: " + str(t1b-t1a))
print("Wilson action square loop expectation " + str(wilson_square_loop) + " time: " + str(t1d-t1c))
print("Wilson action rectangular loop expectation " + str(wilson_rectangular_loop) + " time: " + str(t1f-t1e))

print("Improved action acceptance: " + str(acceptance2) + " total time: " + str(t2b-t2a))
print("Improved action square loop expectation " + str(improved_square_loop) + " time: " + str(t2d-t2c))
print("Improved action rectangular loop expectation " + str(improved_rectangular_loop) + " time: " + str(t2f-t2e))

del lattice

Wilson action acceptance: 0.4707756347656155 total time: 823.3715784549713
Wilson action square loop expectation 0.276387324645 time: 0.559319257736206
Wilson action rectangular loop expectation 0.0776650895487 time: 1.3092007637023926
Improved action acceptance: 0.2631679687499847 total time: 2280.2800352573395
Improved action square loop expectation 0.54374568688 time: 0.651191234588623
Improved action rectangular loop expectation 0.287486892241 time: 1.6563169956207275
